# A Recipe for 'Quick' Computation of two-point correlation function

First of all, we take galaxy data from a redshift survey. In this paper,
we take DR72 VAGC from Kazin *et. al.* [@kazin2010baryonic] to
demonstrate the procedure. Typical data from a redshift survey contains
list of galaxies with their observed redshift, Right ascension (RA) and
declination angles (DEC) that provide the position of each galaxy.
Often, a value added catalog limits the magnitude and chooses a specific
type of galaxy such as Luminous Red Galaxies - LRGs in DR72 VAGC. There
are further parameters in a typical catalog such as magnitude, survey
completeness, radial weights etc. In this paper, we shall only make use
of the redshift and angular position of each galaxy from the catalog.
Survey geometry and distribution are taken in mangle polygon file
formats (`.ply`) provided by the respective survey data release. Value
added catalogs such as the ones taken from (non-official) SDSS DR72
[@dr72vagc] also provide random catalogs. Typically much larger than the
galaxy catalog. The procedure provided in this paper can certainly be
run on these random catalogs. However, as we will see later to reduce
computational effort we can create a smaller random catalog without much
loss of accuracy in the final 2pcf result. \[fig2\] gives a flowchart of
the recipe.

<object data="http://yoursite.com/the.pdf" type="application/pdf" width="700px" height="700px">
    <embed src="http://yoursite.com/the.pdf">
        This browser does not support PDFs. Please download the PDF to view it: <a href="http://yoursite.com/the.pdf">Download PDF</a>.</p>
    </embed>
</object>

![Flow chart of the quick computation recipe](flowchart1 "fig:")
\[flowchart1\]

Data Preparation
----------------

-   Install relevant packages such as `numpy`, `sklearn`, `healpy` and
    `pymangle` using `pip` or `conda` package managers.

-   Take any existing large scale structure catalogs or create one
    yourself by following instructions from (SDSS lss catalog creation
    cite)

-   Calculate Comoving distance as per the model of your choice
    (fiducial cosmology)

-   Load data in RAM as 3xNd matrix \[s,rar,decr\] (Can choose to pickle
    the data or store as ascii file -pickling might not be the best way
    for large data files)

Random Catalog Preparation
--------------------------

-   If we use the random catalog given by the survey. calculation of
    random-random correlation is similar to the steps above. However,
    often it is computationally intensive to calculate for high no. of
    random points. So, a method to create a smaller random catalog which
    can give reasonably accurate results in comparison to the standard
    random catalog.

-   All the major surveys provide the survey geometry in form of angular
    masks in `fits` and/or `ply` formats. We use mangle polygons
    [@hamilton2004scheme] [@swanson2008methods] given by the survey to
    create random points.

-   pymangle is a python wrapper to a faster `C/C++` code to manipulate
    mangle angular masks[@pymangle]. It can create a random catalog with
    a specified no. of data points that follow the angular masks
    provided by the survey. (One can also specify boundaries to
    calculate random points faster and use additional options such as
    polygon weights). In this paper, we demonstrate a simple application
    without weights etc.

-   mangle gives random points in the angular domain only providing RA
    and DEC for the random points. We will also need redshift values to
    be assigned to these random points. These redshifts need to follow
    the survey radial distribution to creating a matching random
    catalog. If we are creating a random catalog that is of the same
    size as the data catalog then it is easier to re-use the redshifts
    of the data and shuffle them to assign to the random catalog points.
    This ensures the radial distribution of the random points to be the
    same as galaxy catalog.

-   If we are to create a random catalog of different size (recommended
    to use bigger($\geq2x$) than the data catalog to reduce the shot
    noise.) we can plot the histogram or Kernel density estimator of the
    data redshift distribution and create a random catalog that mimics
    this distribution. As KDE is a better choice than using a histogram
    (reference), we create a random distribution points for redshifts
    using the same.

Healpix Visualization
---------------------

-   HealPix (and the wrapper healpy) are extremely useful to visualize
    the surveys - Plot surveys

-   After creating the random catalog it is good to visually cross-check
    if they all indeed follow the supplied geometry. Using Healpix we
    can draw the random catalog with random point positions as per the
    mask as shown below in \[fig3\].

    ![Generated random catalog with DR72
    mask[]{data-label="fig3"}](plots/rand100k "fig:")\[h\]

Defining distance metric and creating `BinaryTrees`
---------------------------------------------------

-   Define custom metric in Cython (to improve the speed of
    computation) - metric not to be confused with Cosmology metric. This
    is to find distance between two points in the galaxy survey. This
    depends on the geometry of the Universe (see theory- give citation
    to Matsubara 2004) Also write formula for calculation of angle
    between two points on a sphere. Use square distance to calculate
    faster (reduce sqrt operation) - One needs to take into account that
    the formulae are given for $c=1$ and so to get dimensionless numbers
    inside $\sin$ or $\sinh$ we need to multiply the co-moving distance
    with an appropriate factor assuming the current Hubble expansion
    rate.

-   Create a BallTree with the previously defined custom-metric (Use
    squared distance)

-   Define distance bins for calculating no. of pairs that fall in the
    specified distance radius (Do not forget to use binsq instead of
    bins)

-   Experiment with leafsize option to improve computation
    time/efficiency (ref: jakedvp benchmarking website ref) - talk about
    finding points vs. calculation of distance by brute-force

Calculation of 2pcf
-------------------

-   Calculation of data-data pair correlation is done by `np.diff` to
    find the number of galaxies in the specific bins. This gives us $DD$
    value for the bins.

-   Repeat the same procedure for calculating $DD$ with random catalog
    to obtain $RR$ (random–random correlation)

-   To calculate data-random correlation, we need to find the random
    points in the data-tree or vice-versa. This gives us $DR$ ($=RD$)

-   Having obtained $DD$, $RR$ and $DR$, using scaling factor
    ($N_r/N_d$) we can find the correlation function using the
    Landy-Szalay estimator [@landy1993bias] as We use Landy &
    Szalay (1993) algorithm to get the two-point correlation function,
    $$\xi (s ;z)=1+\left(\frac{N_d}{N}\right)^2\frac{DD}{RR}-2\left(\frac{N_d}{N}\right)\frac{DR}{ RR}$$

    where DD, RR, DR are the number of data–data, random–random,
    data–random and random–data pairs
